In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%run core.py

No files removed: no files as of the day requested


C:\Users\ssu\Documents\github\alerting\process_data.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].apply(lambda x: '&#36;' + '{:,}'.format(x) if x >= 0 else '-&#36;' + '{:,}'.format(abs(x)))
C:\Users\ssu\Documents\github\alerting\insights.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  process['topline'] = process.apply(lambda row: ('Account ' + change + ' <b>' + row['rev_diff_agst_pd_cur'] + '</b> agst prior day.') if row['is_deep_dive'] == 1 else ('The account ' + change + ' aganst 4(7)-day average. If expecte

No files removed: no files as of the day requested


In [ ]:
pd.set_option('max_colwidth',180)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### DSP ###

In [ ]:
	data = PPData(account_types['dsp'])
	#removed dated data
	data.remove_data()
	#generate new data
	filename = account_types['dsp'] + "_" + change_types['jumpers']
	alertee_df = data.get_data(filename)
	processed = ProcessData(alertee_df)
	alertee = processed.get_alertee(account_types['dsp'], alert_params['pct_cutoff'], alert_params['absolute_cutoff'], alert_params['account_rev_threshold'], jumper=True)
	ids, deep_dive_ids, accountnames, table_headers, table_value = processed.multiple_tables(alertee)

In [ ]:
alertee.head()

In [ ]:
	def topline(df, jumper=False):
		process = process_data(df).format_data(df)
		if jumper is False:
			change = 'dropped'
		else: 
			change = 'increased'
		process['topline'] = process.apply(lambda row: ('Account ' + change + ' by ' + row['rev_diff_agst_pd_cur'] + ' agst prior day.') if row['is_deep_dive'] == 1 else ('The account ' + change + ' aganst 4(7)-day average. If expected, ignore the alert. O/w, please take a look.'), axis = 1)
		return process['topline']

In [ ]:
x = topline(alertee, jumper=True)

In [ ]:
x

In [ ]:
ids

In [ ]:
y = ['' for acct in accountnames]
y

In [ ]:
deep_dive_ids

### Pub ###

In [ ]:
pub_df = pd.read_pickle("pub_droppers_20170715.pkl")
pub_processed = ProcessData(pub_df)
pub_tops = pub_processed.top_by_pct_multi_accts(alert_params['dsp_pub_top_changers_by_pct'], positive=False)

In [ ]:
pub_df[pub_df['accountid'] == 534889]

In [ ]:
tops = pub_processed.top_by_pct_multi_accts(0.6, positive=False)

In [ ]:
insight = Insights(tops)
pub_res = insight.top_changers(ids, 'publisher', links['Tableau_dsp_pub'])
pub_res

### Campaigns ###

In [ ]:
lan = pp_data(dsp_queries['campaign'])
#df2 = lan.get_data("pubs_jumper", deep_dive_ids)
df2 = lan.get_data("campaign_for_dropper", deep_dive_ids)

In [ ]:
lan_processed = process_data(df2)

In [ ]:
lan_tops = lan_processed.top_by_pct_multi_accts(1, positive=False)

In [ ]:
insight2 = insights(lan_tops)
cam_res = insight2.top_changers(ids, 'campaign', default_message['campaign'])

In [ ]:
cam_res

### Channel ###

In [ ]:
cn = pp_data(dsp_queries['channel'])
df3 = cn.get_data("channel_dropper", deep_dive_ids)
df3.head()

In [ ]:
insight3 = insights(df3)
cn_res = insight3.facts(ids, 'channel', default_message['channel'])

In [ ]:
cn_res

In [ ]:
email_content = {
    "accountname": accountnames,
    "message1": pub_res,
    "message2": cam_res,
    "message3": cn_res,
    "theader": table_headers,
    "tvalue": table_value
    
}

In [ ]:
email_content['accountname']

In [ ]:
email_temp = EmailAlert("dsp", data_directory+'input.mjml', email_content)

In [ ]:
email_temp.render_template()

In [ ]:
acct_new = acct.groupby(['rev_change_binary']).agg({"publisherid": "count", "difference": sum})

In [ ]:
df.to_csv("pubs_20170706.csv", index = False)

In [ ]:
acctid

In [ ]:
acctids = ', '.join([str(i) for i in acctid])
acctids

In [ ]:
lp = load_data(query['LANDINGPAGE'], 'dsp_landingpage', acctids, 1)

In [ ]:
res = []
for acct in acctid_array:
    print(acct)
    acct_df = lp[lp['buyerid'] == acct]
    acct_top = get_top(acct_df, message['top_3'], 'difference', asc=True)
    adj_dollar_sign_for_negative_number(acct_top, 'difference')
    acct_top['delta'] = np.vectorize(percent)(acct_top['delta'])
    print(acct_top)
#     adj_dollar_sign_for_negative_number(acct_top, difference)
#     acct_top[delta] = np.vectorize(percent)(acct_top[delta])
#     string = strheader
#     if add_link is False:
#         selectedCols = acct_top[[dimension, difference, delta]]	
#     else:
#         selectedCols_ext = acct_top[[dimension, difference, delta, 'dsp']]
#         selectedCols_ext[dimension] = "<a href='" + message['Tableau_dsp_pub'] + "dsp=" + selectedCols_ext['dsp'] + "&publisher=" + selectedCols_ext[dimension] + "' >" + selectedCols_ext[dimension] + '</a>'	
#         selectedCols = selectedCols_ext[[dimension, difference, delta]]
#     for i in range(len(selectedCols)):
#         val = ', '.join([str(i) for i in selectedCols.iloc[i]])
#         string = string + val + ";  "
#     res.append(string)
# return res 

In [ ]:
a = is_dropped(-2.1, -3.6, 1.6)
a

In [ ]:
possible_cause_threshold = -10

def is_dropped(metric1, metric2, metric3):
    if (metric1 < possible_cause_threshold or metric2 < possible_cause_threshold or metric3 < possible_cause_threshold):
        return True 
    else: 
        return False 
    
def is_increased(metric1, metric2, metric3):
    if (metric1 > -possible_cause_threshold or metric2 > -possible_cause_threshold or metric3 > -possible_cause_threshold):
        return True 
    else: 
        return False 
    
def is_negative(metric1, metric2, metric3):
    if (metric1 < 0 or metric2 < 0 or metric3 < 0):
        return True 
    else: 
        return False 

def is_stable(metric1, metric2, metric3):
    if (metric1 < 5 and metric1 > -5) and (metric2 < 5 and metric2 > -5) and (metric3 < 5 and metric3 > -5):
        return True 
    else: 
        return False 

def get_summary(offer_pd, offer_4d, offer_7d, offerrate_pd, offerrate_4d, offerrate_7d, bidrate_pd, bidrate_4d, bidrate_7d, winrate_pd, winrate_4d, winrate_7d, revcpm_change_agst_pd, revcpm_change_agst_4davg, revcpm_change_agst_7davg, costcpm_change_agst_pd, costcpm_change_agst_4davg, costcpm_change_agst_7davg):
    if (is_dropped(offer_pd, offer_4d, offer_7d) is True or is_dropped(offer_pd, offer_4d, offer_7d) is True) and (is_dropped(bidrate_pd, bidrate_4d, bidrate_7d) is False and is_dropped(winrate_pd, winrate_4d, winrate_7d) is False):
        return "most likely due to supply change. Please check rev-driving pubs' supply, DSP throttle changes such as PQS, domain/pub block etc."
    elif (is_dropped(bidrate_pd, bidrate_4d, bidrate_7d) is True or is_dropped(winrate_pd, winrate_4d, winrate_7d) is True) and (is_dropped(offer_pd, offer_4d, offer_7d) is False and is_dropped(offer_pd, offer_4d, offer_7d) is False): 
        return "most likely due to demand change. Please check if drop is expected."
    elif (is_stable(offer_pd, offer_4d, offer_7d) is True and is_stable(offer_pd, offer_4d, offer_7d) is True) and is_dropped(winrate_pd, winrate_4d, winrate_7d) is True and is_increased(costcpm_change_agst_pd, costcpm_change_agst_4davg, costcpm_change_agst_7davg) is True and is_stable(revcpm_change_agst_pd, revcpm_change_agst_4davg, revcpm_change_agst_7davg) is True:
        return "most likely due to intenser competition in upstream auction or change on pub's floor"
    elif (is_dropped(offer_pd, offer_4d, offer_7d) is True or is_dropped(offer_pd, offer_4d, offer_7d) is True) and (is_dropped(bidrate_pd, bidrate_4d, bidrate_7d) is True or is_dropped(winrate_pd, winrate_4d, winrate_7d) is True):
        return "most likely due to both supply drop and demand drop. Please check both sides."
    else:
        return "could be due to multiple reasons. Please check all metrics."
    

In [ ]:
test['highlight'] = np.vectorize(get_summary)(test['offer_change_agst_pd'], test['offer_change_agst_4davg'], test['offer_change_agst_7davg'], test['offerrate_change_agst_pd'], test['offerrate_change_agst_4davg'], test['offerrate_change_agst_7davg'], test['bidrate_change_agst_pd'], test['bidrate_change_agst_4davg'], test['bidrate_change_agst_7davg'], test['winrate_change_agst_pd'], test['winrate_change_agst_4davg'], test['winrate_change_agst_7davg'], test['revcpm_change_agst_pd'], test['revcpm_change_agst_4davg'], test['revcpm_change_agst_7davg'], test['costcpm_change_agst_pd'], test['costcpm_change_agst_4davg'], test['costcpm_change_agst_7davg'])

In [ ]:
test.head()

In [ ]:
test.dtypes

In [ ]:
buyerid = test['buyerid'].T.drop_duplicates().T
y = np.array(buyerid['buyerid'])
param = ', '.join([str(i) for i in y])
param

In [ ]:
bl = load_data(query['BIDLOSS'], "dsp_bidloss", param, 1)
bls = bl[bl['buyerid'] == 547259]

In [ ]:
top = get_top(bls, 3, "impressions", asc=False )

In [ ]:
top

In [ ]:
dsp_pub = load_data(query['DSP-PUB'], "dsp_pub", param, 2)

In [ ]:
dsp_pub.head()

In [ ]:
dp = dsp_pub.sort_values(by = ['buyerid', 'rev_difference'], ascending = [True, True])

In [ ]:
pub = get_topdrop(param, query['DSP-PUB'], 'dsp_pub', 2, 'publisher','rev_difference', 'rev_delta', '* Top drop pubs: ')

In [ ]:
pubs = get_topdrop(y, dp, 'publisher','rev_difference', 'rev_delta', 'Top drop pubs: ' )
pubs

In [ ]:
lan = load_data(query['LANDINGPAGE'], "dsp_landingpage", param, 1)
#lan = lan.sort_values(by = ['buyerid', 'difference'], ascending = [True, True])

In [ ]:
lan.head()

In [ ]:
lans = get_topdrop(y, lan, 'landingpagedomain','difference', 'delta', 'Top drop campaigns: ' )
lans

In [ ]:
dc = load_data(query['DC'], "dsp_dc", param, 1)

In [ ]:
dcs = get_topdrop(y, dc, 'dc','difference', 'delta', 'Change by DC: ' )
dcs

In [ ]:
cn = load_data(query['CHANNEL'], "dsp_channel", param, 1)

In [ ]:
cns = get_topdrop(y, cn, 'channel','difference', 'delta', 'Change by Channel: ' )
cns

In [ ]:
def load_data(sql, fname, param=None):
    with open(sql) as f:
        if param==None:
            query = f.read().strip()
        else:
            query = f.read().strip() % (param, param)
    filename = os.path.join(data_directory, "{}_{}.pkl".format(fname, datetime.datetime.now().strftime("%Y%m%d")))
    if os.path.exists(filename) is False:
        df = pd.read_sql(query, create_engine("impala://impala.pulse.prod"))
        df.to_pickle(filename)
    else:
        df = pd.read_pickle(filename)
    return df

In [ ]:
def cal_supply_demand_relationship(acctid):
    acct = dp[dp['buyerid'] == acctid]
    a = np.array(acct['rev_delta'])
    b = np.array(acct['offer_delta'])
    r = np.corrcoef(a,b)[1, 0]
    print(r)

In [ ]:
for i in y:
    print(i)
    cal_supply_demand_relationship(i)

In [ ]:
	day = df['day'][0]
	accts = pd.Series(df.index)
	headers = []
	value = []
	yesterday_cols = ['revenue' , 'offer' , 'offerrate' , 'offermatchrate' , 'bidrate' , 'blockrate' , 'winrate' , 'revcpm'
	                     , 'costcpm' , 'margin' , 'timeoutrate']
	yesterday = df[yesterday_cols]
	yesterday_T = yesterday.T
	delta_col = list(df.loc[:, 'revenue_change_agst_pd':'timeoutrate_change_agst_7davg'])
	delta = df[delta_col]

In [ ]:
	for acct in accts:
		acct_yesterday = yesterday_T[acct]
		pday = slice_data('pd', delta, acct)
		avg4d = slice_data('4davg', delta, acct) 
		avg7d = slice_data('7davg', delta, acct) 
		acct_data = pd.concat([acct_yesterday, pday, avg4d, avg7d], axis = 1, keys=[day, 'change agst prior day', 'change agst 4-day avg', 'change agst 7-day avg'])
		acct_data= acct_data.reset_index()
		data = df_json(acct_data)
		header = data['columns']
		val = data['data']
		headers.append(header)
		value.append(val)

In [ ]:
acct_data

In [ ]:
acct_data['metric'] = acct_data['metric'].apply(lambda x: x.replace('rate', ' rate'))

In [ ]:
acct_data

In [ ]:
x = '-hello'

In [ ]:
y = x.split(" ",1)
y

In [ ]:
message = {
    'top_n': 3,
    'dsp_pub_Tableau_link': 'http://tableau:8000/#/views/DSPTrendsDashboard/DSPPublisherTrends7D?'
}

In [ ]:
message['top_n']

In [ ]:
test = pd.read_pickle("dsp_landingpage_20170522.pkl")

In [ ]:
def test_var_args(farg, *args):
    print("formal arg:", farg)
    for arg in args:
        print("another arg:", arg)

test_var_args(1, "two", 3)